In [1]:
import numpy as np
import keras.models
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
model = keras.models.load_model('face_mask_detection_model.h5')

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 92, 92, 112)       8512      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 112)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 48)        134448    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 48)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 21168)             0         
                                                                 
 dense (Dense)               (None, 64)                1

In [11]:
import cv2
vid = cv2.VideoCapture(0)
address = 'http://192.168.0.102:8080/video'
vid.open(address)
# vid.set(cv2.CAP_PROP_FRAME_WIDTH, 160)
# vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 120)

True

In [12]:
import imutils
while(True):
    # Capture the video frame
    # by frame
    ret, img = vid.read()
#     img = imutils.resize(img, width=1000)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(gray, minSize=(200,200))
    
    for (x, y, w, h) in faces:
        face_img = img[y:y + h, x:x + w]

        face_img = cv2.resize(face_img, (96,96))
        face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face_arr = np.asarray(face_img)
        face_arr = face_arr / 255.0
        res = model.predict(np.array([face_arr]))[0]
        if res[0] > res[1]:
            color = (0, 0, 255)
            label = 'UnMasked: ' + "{:.2f}".format(res[0]*100)
        else:
            color = (0, 255, 0)
            label = 'Masked: ' + "{:.2f}".format(res[1]*100)
        
        x1, y1, x2, y2 = x, y, x+w, y+h
        text_color = (255, 0, 0)
        # For bounding box
        img = cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

        # For the text background
        # Finds space required by the text so that we can put a background with that amount of width.
        (w, h), _ = cv2.getTextSize(
                label, cv2.FONT_HERSHEY_SIMPLEX, 1, 1)

        # Prints the text.    
        img = cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), color, -1)
        img = cv2.putText(img, label, (x1, y1),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                            
    cv2.imshow('image', img)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [10]:
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

In [ ]:
image = cv2.imread('image_2.jpeg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

faceCascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
faces = faceCascade.detectMultiScale(gray,1.1,3,minSize=(200,200))

print("[INFO] Found {0} Faces!".format(len(faces)))

for (x, y, w, h) in faces:
    face_img = image[y:y + h, x:x + w]
    print(w, ' ', h)
    face_img = cv2.resize(face_img, (96,96))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_arr = np.asarray(face_img)
    
    plt.imshow(face_arr)
    plt.show()
    
    res = model.predict(np.array([face_arr]))[0]
    print(res)
    if res[0] > res[1]:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
    else:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    